In [ ]:
import cv2
import mediapipe as mp
import math
import numpy as np
import time

# Impor pustaka 3D
from OpenGL.GL import *
from OpenGL.GLUT import *
import glfw

# --- Inisialisasi ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

cap = cv2.VideoCapture(0)

# Variabel untuk Zoom & Rotate
initial_distance = None  
initial_angle = None
zoom_level = 1.0        
rotation_angle = 0.0

# Variabel untuk Posisi
position_x = 0.0
position_y = 0.0

# Variabel untuk Cooldown
last_zoom_time = time.time()
ZOOM_COOLDOWN = 2.0  # 2 detik

# Variabel untuk Status
is_zooming = True
MIN_ZOOM = 0.5
MAX_ZOOM = 2.0

# --- Fungsi untuk Mendeteksi Kepalan Tangan ---
def is_fist(hand_landmarks):
    # Mengukur jarak antara ujung jari dan sendi di bawahnya
    distance_index = math.hypot(
        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x - hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].x,
        hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y - hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP].y
    )
    distance_middle = math.hypot(
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x - hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].x,
        hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y - hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_PIP].y
    )
    # Anda bisa tambahkan jari lain untuk akurasi yang lebih baik
    # Tentukan ambang batas (threshold) jarak
    if distance_index < 0.05 and distance_middle < 0.05:
        return True
    return False

# --- Fungsi untuk Mengatur OpenGL ---
def setup_gl(width, height):
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glOrtho(-1.0, 1.0, -1.0, 1.0, -1.0, 1.0)

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

# Fungsi untuk memuat file OBJ
def load_obj(filename):
    vertices = []
    faces = []
    for line in open(filename, 'r'):
        if line.startswith('v '):
            vertices.append(list(map(float, line.strip().split()[1:4])))
        elif line.startswith('f '):
            face_indices = line.strip().split()[1:]
            faces.append([int(i.split('/')[0]) - 1 for i in face_indices])
    return vertices, faces

try:
    
    vertices, faces = load_obj('Rubik.obj')
except FileNotFoundError:
    print("File 3D tidak ditemukan! Pastikan 'your_model.obj' berada di folder yang sama.")
    exit()

def draw_obj():
    glBegin(GL_TRIANGLES)
    glColor3f(0.8, 0.8, 0.8) # Warna objek
    for face in faces:
        for vertex_index in face:
            glVertex3fv(vertices[vertex_index])
    glEnd()

# --- Loop Utama ---
if not glfw.init():
    exit()

window = glfw.create_window(1280, 720, "Hand Gesture 3D Control", None, None)
if not window:
    glfw.terminate()
    exit()

glfw.make_context_current(window)
setup_gl(1280, 720)

while not glfw.window_should_close(window):
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    result = hands.process(frame_rgb)
    
    current_time = time.time()

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            h, w, c = frame.shape
            
            # Mendeteksi kepalan tangan
            if is_fist(hand_landmarks):
                is_zooming = False
            else:
                is_zooming = True
                
            # Logika zoom hanya jika bukan kepalan tangan DAN cooldown sudah selesai
            if is_zooming and (current_time - last_zoom_time) > ZOOM_COOLDOWN:
                # === Logika Zoom (Jari Telunjuk & Jempol) ===
                index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
                index_x, index_y = int(index_tip.x * w), int(index_tip.y * h)
                thumb_x, thumb_y = int(thumb_tip.x * w), int(thumb_tip.y * h)
                current_distance = math.hypot(index_x - thumb_x, index_y - thumb_y)
                
                if initial_distance is None:
                    initial_distance = current_distance
                
                # Menghitung zoom dan membatasinya
                zoom_level = min(MAX_ZOOM, max(MIN_ZOOM, current_distance / initial_distance))
                
                # Reset initial_distance dan cooldown
                last_zoom_time = current_time
            
            # === Logika Rotasi (Pergelangan & Jari Tengah) ===
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            wrist_x, wrist_y = int(wrist.x * w), int(wrist.y * h)
            middle_x, middle_y = int(middle_tip.x * w), int(middle_tip.y * h)
            angle_rad = math.atan2(middle_y - wrist_y, middle_x - wrist_x)
            angle_deg = math.degrees(angle_rad) + 90
            if initial_angle is None:
                initial_angle = angle_deg
            rotation_angle = angle_deg - initial_angle
            if rotation_angle > 180:
                rotation_angle -= 360
            elif rotation_angle < -180:
                rotation_angle += 360

            # === Logika Posisi ===
            hand_center_x = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x
            hand_center_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
            position_x = (hand_center_x * 2) - 1.0 
            position_y = -((hand_center_y * 2) - 1.0)
    
    # === Rendering OpenGL ===
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    glTranslatef(position_x, position_y, 0.0)
    glRotatef(rotation_angle, 0, 0, 1)
    glScalef(zoom_level, zoom_level, zoom_level)
    draw_obj()
    glfw.swap_buffers(window)
    glfw.poll_events()

cap.release()
glfw.terminate()

File 3D tidak ditemukan! Pastikan 'your_model.obj' berada di folder yang sama.


NameError: name 'faces' is not defined